### RFM Value

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
df= pd.read_csv('C:/Users/Administrator/Downloads/Clevertap_Date_wise.csv',encoding = "ISO-8859-1")
df['Date']=pd.to_datetime(df['Date'])
df1=df.fillna(0)
m=df1.groupby(['AccountName'])['Count'].aggregate('sum').reset_index().sort_values('Count', ascending=False)
f=df1.groupby(['AccountName'])['Count'].aggregate('count').reset_index().sort_values('Count', ascending=False)
recency=pd.DataFrame(df1.groupby(['AccountName'])['Date'].max())
recency['Company']=recency.index
recency.reset_index(inplace=True)
df2=pd.concat([m,f['Count'],recency['Date']],axis=1)
df2.columns=['AccountName','Monetory','Frequency','Recency']
df2['Date'] = pd.to_datetime(df2['Recency'])
import datetime
now = pd.to_datetime(datetime.date.today())
df2['Recency']=abs(df2['Date']-now)
df2.drop('Date',axis=1,inplace=True)
quantiles = df2.quantile(q=[0.20,0.4,0.6,0.8])
quantiles = quantiles.to_dict()
segmented_rfm = df2
segmented_rfm['Recency']=pd.DataFrame((segmented_rfm['Recency']/(24*60*60*1000000000)).astype('int64'))
def RScore(x,p,d):
    if x <= d[p][0.20]:
        return 5
    elif x <= d[p][0.40]:
        return 4
    elif x <= d[p][0.60]: 
        return 3
    elif x <= d[p][0.80]: 
        return 2
    else:
        return 1
    
def FMScore(x,p,d):
    if x <= d[p][0.20]:
        return 1
    elif x <= d[p][0.40]:
        return 2
    elif x <= d[p][0.60]: 
        return 3
    elif x <= d[p][0.80]: 
        return 4
    else:
        return 5

In [132]:
quantiles=segmented_rfm.quantile(q=[0.20,0.4,0.6,0.8])

In [133]:
quantiles

,Monetory,Frequency,Recency,r_quartile,f_quartile,m_quartile
0.2,2.0,2.0,1.0,2.0,1.0,1.0
0.4,11.0,6.0,6.0,3.0,2.0,2.0
0.6,65.4,32.0,54.0,4.0,3.0,3.4
0.8,364.8,113.2,140.0,5.0,4.2,4.2


In [9]:
quantile=quantiles.reset_index()

### Recency logic

In [12]:
segmented_rfm['r_quartile']=segmented_rfm['Recency']

In [14]:
for i in range(len(segmented_rfm['Recency'])):
    if (segmented_rfm['Recency'][i]<=quantile['Recency'][0]):
        segmented_rfm['r_quartile'][i]=5
    elif (segmented_rfm['Recency'][i]<=quantile['Recency'][1]):
        segmented_rfm['r_quartile'][i]=4
    elif (segmented_rfm['Recency'][i]<=quantile['Recency'][2]):
        segmented_rfm['r_quartile'][i]=3
    elif (segmented_rfm['Recency'][i]<=quantile['Recency'][3]):
        segmented_rfm['r_quartile'][i]=2
    else:
        segmented_rfm['r_quartile'][i]=1

In [16]:
segmented_rfm['f_quartile'] = segmented_rfm['Frequency'].apply(FMScore, args=('Frequency',quantiles,))
segmented_rfm['m_quartile'] = segmented_rfm['Monetory'].apply(FMScore, args=('Monetory',quantiles,))

In [22]:
segmented_rfm['RFMScore'] = segmented_rfm.r_quartile.map(str) + segmented_rfm.f_quartile.map(str) + segmented_rfm.m_quartile.map(str)

,AccountName,Monetory,Frequency,Recency,r_quartile,f_quartile,m_quartile,RFMScore
960,Systemic Electric Pvt Ltd,601,68,1,5,4,5,545


In [134]:
quantile

,index,Monetory,Frequency,Recency
0,0.2,2.0,2.0,1.0
1,0.4,11.0,6.0,6.0
2,0.6,65.4,32.0,54.0
3,0.8,364.8,113.2,140.0


In [162]:
for i in range(len(segmented_rfm)):
    if((segmented_rfm['r_quartile'][i]==5) & (segmented_rfm['f_quartile'][i]==5) & (segmented_rfm['m_quartile'][i]==5)):
        segmented_rfm['Label'][i]='Champion'
    elif((segmented_rfm['r_quartile'][i]==5) & ((segmented_rfm['f_quartile'][i]==4) | (segmented_rfm['f_quartile'][i]==5)) & ((segmented_rfm['m_quartile'][i]==4) | (segmented_rfm['m_quartile'][i]==5))):
        segmented_rfm['Label'][i]='Loyal Customers'
    elif((segmented_rfm['r_quartile'][i]==5) & ((segmented_rfm['f_quartile'][i]==4) | (segmented_rfm['f_quartile'][i]==5)) & ((segmented_rfm['m_quartile'][i]==4) | (segmented_rfm['m_quartile'][i]==5))):
         segmented_rfm['Label'][i]='Potential loyalist'
    elif(((segmented_rfm['r_quartile'][i]==5)|(segmented_rfm['r_quartile'][i]==4)) & (segmented_rfm['f_quartile'][i]<=1) & (segmented_rfm['m_quartile'][i]<=1)):   
        segmented_rfm['Label'][i]='New customers'
    elif(((segmented_rfm['r_quartile'][i]==3) | (segmented_rfm['r_quartile'][i]==4)) & (segmented_rfm['f_quartile'][i]<=1) & (segmented_rfm['m_quartile'][i]<=1)):
        segmented_rfm['Label'][i]='Promsing'
    elif(((segmented_rfm['r_quartile'][i]==2) | (segmented_rfm['r_quartile'][i]==3)) & ((segmented_rfm['f_quartile'][i]==2) | (segmented_rfm['f_quartile'][i]==3)) & ((segmented_rfm['m_quartile'][i]==2) | (segmented_rfm['m_quartile'][i]==3))):
        segmented_rfm['Label'][i]='Need Attention'
    elif((segmented_rfm['r_quartile'][i]<=2)  & ((segmented_rfm['f_quartile'][i]>=2) & (segmented_rfm['f_quartile'][i]<=5)) & ((segmented_rfm['m_quartile'][i]>=2) & (segmented_rfm['m_quartile'][i]<=5))):
        segmented_rfm['Label'][i]='At Risk'
    elif((segmented_rfm['r_quartile'][i]<=3)  & ((segmented_rfm['f_quartile'][i]==4) | (segmented_rfm['f_quartile'][i]==5)) & ((segmented_rfm['m_quartile'][i]==4) | (segmented_rfm['m_quartile'][i]==5))):
        segmented_rfm['Label'][i]='Can’t Lose Them'
    elif((segmented_rfm['r_quartile'][i]==1)&(segmented_rfm['f_quartile'][i]<=5)&(segmented_rfm['m_quartile'][i]<=5)):
        segmented_rfm['Label'][i]='Lost'

In [163]:
segmented_rfm['Label'].value_counts()

Loyal Customers       196
At Risk               189
Champion              178
Need Attention        171
Lost                   91
Hibernating            86
Potential loyalist     85
Promsing               64
Can’t Lose Them        47
New customers          18
Name: Label, dtype: int64

In [164]:
segmented_rfm.to_csv('Segmentation.csv')